In [3]:
import json
from azure.eventhub import EventHubProducerClient, EventData
import os
import socket
import random

from random import randrange


eventHubConnString = "Endpoint=sb://a-hroy72qhfmv4k-ehub-ns.servicebus.windows.net/;SharedAccessKeyName=analytics-demo;SharedAccessKey=LEag6c1sQIF8uiFAsQ1qfBA9y2VrZl0Nw+AEhMVDQIw=" 
eventHubNameevents = "events"

producer_events = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameevents)

hostname = socket.gethostname()

In [14]:
from faker import Faker
from enum import Enum

# class syntax
class EVENT_TYPE(Enum):
    CLICK = 1
    IMPRESSION = 2

productIds = [707,708,711,712,714,715,716,717,718,722,738,739,742,743,747,748,779,780,781,782,783,784,792,793,794,795,796,797,798,799,800,801,808,809,810,813,822,835,836,838,858,859,860,864,865,867,868,869,870,873,874,875,876,877,880,881,883,884,885,886,889,891,892,893,894,895,896,899,900,904,905,907,908,909,910,913,916,917,918,920,924,925,926,935,936,937,938,939,940,944,945,947,948,949,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,996,997,998,999]


def generateImpressionEvent(isAnomaly, productId):
    faker = Faker()

    event = {}
    event["eventType"] = EVENT_TYPE.IMPRESSION.name
    event["eventID"] = faker.uuid4()
    event["eventDate"] = faker.date_time_between(start_date="-1w", end_date="now").isoformat()
    if productId:
        event["productId"] = str(productId)
    else:
        event["productId"] = faker.random_element(productIds)

    randomizeUnsupported = randrange(100)
    userAgent = {}
    userAgent["platform"] = faker.random_element(["Windows", "Mac", "Linux", "iOS", "Android"])
    if randomizeUnsupported <2:
        userAgent["browser"] = "Unsupported"
    else:
        if userAgent["platform"] == "Windows":
            userAgent["browser"] = faker.random_element(["Edge", "Chrome", "Firefox", "Safari"])
        elif userAgent["platform"] == "Mac":
            userAgent["browser"] = faker.random_element(["Chrome", "Firefox", "Safari"])
        elif userAgent["platform"] == "Linux":
            userAgent["browser"] = faker.random_element(["Chrome", "Firefox"])
        elif userAgent["platform"] == "iOS":
            userAgent["browser"] = faker.random_element(["Safari", "Chrome"])
        elif userAgent["platform"] == "Android":
            userAgent["browser"] = faker.random_element(["Chrome", "Firefox"])
        userAgent["browserVersion"] = faker.random_element(["10.2", "13.6", "8.6", "8.5", "11.2", "14.6", "6.6", "4.5"])
    event["userAgent"] = userAgent

    event["device"] = faker.random_element(["mobile", "computer", "tablet", "mobile", "computer"])
    event["ip_address"] = faker.ipv4()

    # only set the referer for CLICK events
    refererPayload = {}
    event["referer"] = refererPayload
    
    if isAnomaly:
        event["page_loading_seconds"] = faker.random_number(4)/100
    else:
        event["page_loading_seconds"] = faker.random_number(4)/1000
    return event


def generateClickEvent(impressionEvent, isAnomaly):
    faker = Faker()

    event = {}
    event["eventType"] = EVENT_TYPE.CLICK.name
    event["eventID"] = faker.uuid4()
    event["eventDate"] = impressionEvent["eventDate"]
    event["productId"] = impressionEvent["productId"]
    event["userAgent"] = impressionEvent["userAgent"]
    event["device"] = impressionEvent["device"]
    event["ip_address"] = impressionEvent["ip_address"]

    # only set the referer for CLICK events, refererPayload differs by campaign type
    refererPayload = {}    
    refererPayload["url"] = faker.uri()
    refererPayload["campaignType"] = faker.random_element( ["organic", "bing", "google", "facebook", "instagram", "twitter", "pinterest", "email", "affiliate"])
    match refererPayload["campaignType"]:
        case "bing"| "google"| "facebook"| "instagram":
            refererPayload["medium"] = "cpc"
            refererPayload["adId"] = faker.uuid4()
            refererPayload["adGroup"] = faker.uuid4()
            refererPayload["adTitle"] = faker.sentence()
        case "twitter":
            refererPayload["medium"] = "cpc"
            refererPayload["adId"] = faker.uuid4()
        case "pinterest":
            refererPayload["medium"] = "cpc"
            refererPayload["adId"] = faker.uuid4()
        case "email":
            refererPayload["medium"] = "email"
            refererPayload["campaignId"] = faker.uuid4()
            refererPayload["emailId"] = faker.email()
        case "affiliate":
            refererPayload["medium"] = "affiliate"
            refererPayload["affiliateId"] = faker.uuid4()
    event["referer"] = refererPayload
    
    if isAnomaly:
        event["page_loading_seconds"] = faker.random_number(4)/100
    else:
        event["page_loading_seconds"] = faker.random_number(4)/1000
    return event

In [ ]:
def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    # event_data_batch = producer.create_batch() 
    # event_data_batch.add(EventData(eventString)) 
    # producer.send_batch(event_data_batch)

def generateEvents(isAnomaly = False, productId = None):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isAnomaly, productId)    
            sendToEventsHub(impressionEvent, producer_events)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent, isAnomaly)    
                sendToEventsHub(clickEvent, producer_events)
    except KeyboardInterrupt:
        producer_events.close()


In [15]:
import datetime
print(datetime.datetime.now())
generateEvents(False, None)
print(datetime.datetime.now())


2024-02-21 13:58:51.590474
{"eventType": "IMPRESSION", "eventID": "df568735-c945-4d41-a82a-998550f35cc2", "eventDate": "2024-02-15T04:28:00", "productId": 985, "userAgent": {"platform": "Linux", "browser": "Chrome", "browserVersion": "6.6"}, "device": "mobile", "ip_address": "188.91.224.138", "referer": {}, "page_loading_seconds": 3.897}
{"eventType": "IMPRESSION", "eventID": "b549eea2-4e06-4739-a207-ca7777a69f52", "eventDate": "2024-02-16T08:18:30", "productId": 715, "userAgent": {"platform": "Linux", "browser": "Firefox", "browserVersion": "4.5"}, "device": "computer", "ip_address": "82.145.61.137", "referer": {}, "page_loading_seconds": 9.689}
{"eventType": "IMPRESSION", "eventID": "f4241199-45b8-4c9d-ba36-06f442f2414e", "eventDate": "2024-02-20T02:27:39", "productId": 869, "userAgent": {"platform": "Linux", "browser": "Firefox", "browserVersion": "14.6"}, "device": "mobile", "ip_address": "97.48.87.189", "referer": {}, "page_loading_seconds": 1.8}
{"eventType": "CLICK", "eventID": 